In [1]:
from Systems import DynamicSystemModel, GaussianNoise
from MomentMatching import TaylorTransform
from MomentMatching.Estimator import Estimator
from ExpectationPropagation.Nodes import build_nodes, node_estimator, node_system
from StateModel import Gaussian
import numpy as np

In [2]:
F = np.array([[3.5]])
H = np.array([[2.2]])

def f(x, t, u=None):
    return F@x

def h(x, t=None, u=None):
    return H@x

class LinearSystem(DynamicSystemModel):
    def __init__(self):
        init_dist = Gaussian(mean_vec=np.array([0.34]), cov_mat=np.eye(1) * 10)
        super().__init__(system_dim=1,
                         measurement_dim=1,
                         transition=f,
                         measurement=h,
                         system_noise=GaussianNoise(dimension=1,
                                                    cov=np.eye(1) * 2),
                         measurement_noise=GaussianNoise(dimension=1,
                                                         cov=np.eye(1) * 3),
                         init_distribution=init_dist
                         )
    # def __init__(self):
    #     init_dist = Gaussian(mean_vec=np.array([0.0]), cov_mat=np.eye(1) * 0.001)
    #     super().__init__(system_dim=1,
    #                      measurement_dim=1,
    #                      transition=f,
    #                      measurement=h,
    #                      system_noise=GaussianNoise(dimension=1,
    #                                                 cov=np.eye(1) * 0.01),
    #                      measurement_noise=GaussianNoise(dimension=1,
    #                                                      cov=np.eye(1) * 0.01),
    #                      init_distribution=init_dist
    #                      )

In [21]:
SEED = 12345
timesteps = 5

power = 0.1

system = LinearSystem()
transition_transform = TaylorTransform(dim=1)
measurement_transform = TaylorTransform(dim=1)


np.random.seed(seed=SEED)
data = system.simulate(timesteps)
x_true, x_noisy, y_true, y_noisy = zip(*data)

In [22]:
# Compute factors explicitly
class ManualFactorUpdate:
    def __init__(self):
        self.F = F
        self.H = H
        self.Q = system.system_noise.cov
        self.R = system.measurement_noise.cov
        
    def measurement_update(self, node):
        y = np.atleast_1d(node.meas)
        H, R = self.H, self.R
        old_cavity = node.marginal / node.measurement_factor
        mean = np.linalg.solve(H, y)
        cov = np.linalg.inv(H.T@np.linalg.solve(R, H))
        node.measurement_factor = Gaussian(mean, cov)
        node.marginal = old_cavity * node.measurement_factor

    def forward_update(self, node):
        F, Q = self.F, self.Q
        old_cavity = node.marginal / node.forward_factor
        try:
            prev_node = node.prev_node.copy()
            back_cavity = prev_node.marginal / (prev_node.back_factor)
        except AttributeError:
            back_cavity = node.prior
        meanxback = back_cavity.mean
        covxback = back_cavity.cov
        mean = F@meanxback
        cov = F@covxback@F.T + Q
        node.forward_factor = Gaussian(mean, cov)
        node.marginal = old_cavity * node.forward_factor

    def backward_update(self, node):
        F, Q = self.F, self.Q
        old_cavity = node.marginal / node.back_factor
        try:
            next_node = node.next_node.copy()
            fwd_cavity = next_node.marginal / (next_node.forward_factor)
            meanxfwd = fwd_cavity.mean
            covxfwd = fwd_cavity.cov
            mean = np.linalg.solve(F, meanxfwd)
            cov = np.linalg.inv(F.T@np.linalg.solve(Q+covxfwd, F))
            node.back_factor = Gaussian(mean, cov)
            node.marginal = old_cavity * node.back_factor
        except AttributeError:
            return

    def __call__(self, nodes):
        for node in nodes:
            self.forward_update(node)
            self.measurement_update(node)
        for node in nodes:
            self.backward_update(node)


def compare_factors(nodes_baseline, nodes_power_ep, power):
    i = 0
    for node_1, node_2 in zip(nodes_baseline, nodes_power_ep):
        print("-------------------")
        print(f"Node {i}")
        print("-------------------")
        fwd_1 = node_1.forward_factor
        fwd_2 = node_2.forward_factor ** (1 / power)
        fwd_mean_diff = np.abs(fwd_1.mean[0] - fwd_2.mean[0])
        fwd_cov_diff = np.abs(fwd_1.cov[0][0] - fwd_2.cov[0][0])
        print(f"Forward factor: Mean difference = {fwd_mean_diff:.4e}, Cov difference = {fwd_cov_diff:.4e}")
        meas_1 = node_1.measurement_factor
        meas_2 = node_2.measurement_factor ** (1 / power)
        meas_mean_diff = np.abs(meas_1.mean[0] - meas_2.mean[0])
        meas_cov_diff = np.abs(meas_1.cov[0][0] - meas_2.cov[0][0])
        print(f"Measurement factor: Mean difference = {meas_mean_diff:.4e}, Cov difference = {meas_cov_diff:.4e}")
        back_1 = node_1.back_factor
        back_2 = node_2.back_factor ** (1 / power)
        back_mean_diff = np.abs(back_1.mean[0] - back_2.mean[0])
        back_cov_diff = np.abs(back_1.cov[0][0] - back_2.cov[0][0])
        print(f"Backward factor: Mean difference = {back_mean_diff:.4e}, Cov difference = {back_cov_diff:.4e}")
        i += 1


In [23]:
estim_1 = Estimator(trans_map=transition_transform,
                meas_map=measurement_transform,
                trans_noise=system.transition_noise.cov,
                meas_noise=system.measurement_noise.cov,
                power=1.,
                damping=1.)

nodes_baseline = build_nodes(N=timesteps, dim=1)
nodes_baseline = node_estimator(nodes=nodes_baseline, estimator=estim_1)
nodes_baseline = node_system(nodes=nodes_baseline, system_model=system, measurements=y_noisy)

# kalman = ManualFactorUpdate()
# kalman(nodes_baseline)

for node in nodes_baseline:
    node.fwd_update()
    node.meas_update()
for node in reversed(nodes_baseline):
    node.back_update()

In [24]:
# Results from EP
estim_2 = Estimator(trans_map=transition_transform,
                meas_map=measurement_transform,
                trans_noise=system.transition_noise.cov,
                meas_noise=system.measurement_noise.cov,
                power=power,
                damping=1.)

nodes = build_nodes(N=timesteps, dim=1)
nodes = node_estimator(nodes=nodes, estimator=estim_2)
nodes = node_system(nodes=nodes, system_model=system, measurements=y_noisy)

for node in nodes:
    node.fwd_update()
    node.meas_update()
for node in reversed(nodes):
    node.back_update()

In [25]:
compare_factors(nodes_baseline, nodes, power)

-------------------
Node 0
-------------------
Forward factor: Mean difference = 1.0707e-12, Cov difference = 8.9978e-07
Measurement factor: Mean difference = 4.5029e-06, Cov difference = 4.3575e-06
Backward factor: Mean difference = 7.5742e-06, Cov difference = 5.0559e-07
-------------------
Node 1
-------------------
Forward factor: Mean difference = 1.5440e-05, Cov difference = 5.3750e-05
Measurement factor: Mean difference = 3.6606e-06, Cov difference = 4.3198e-06
Backward factor: Mean difference = 1.9560e-05, Cov difference = 4.1746e-07
-------------------
Node 2
-------------------
Forward factor: Mean difference = 1.5904e-05, Cov difference = 5.0009e-05
Measurement factor: Mean difference = 3.0972e-05, Cov difference = 4.3162e-06
Backward factor: Mean difference = 7.6132e-05, Cov difference = 4.2415e-07
-------------------
Node 3
-------------------
Forward factor: Mean difference = 1.0414e-04, Cov difference = 4.9739e-05
Measurement factor: Mean difference = 1.0492e-04, Cov dif